<a href="https://colab.research.google.com/github/urmzd/lepus-classifier/blob/main/notebooks/example_bootstrap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!rm -rf /content/rabbit-classifier/
!git clone https://github.com/urmzd/rabbit-classifier
!pip install loguru pipetools pytorch-lightning wandb
!pip install pyyaml==5.4.1
!wandb login

Cloning into 'rabbit-classifier'...
remote: Enumerating objects: 987, done.
remote: Counting objects: 100% (987/987), done.
remote: Compressing objects: 100% (614/614), done.
remote: Total 987 (delta 532), reused 655 (delta 274), pack-reused 0
Receiving objects: 100% (987/987), 52.14 MiB | 10.36 MiB/s, done.
Resolving deltas: 100% (532/532), done.
wandb: Currently logged in as: csci-4155-rabbit-classifier (use `wandb login --relogin` to force relogin)


In [ ]:
import os
os.kill(os.getpid(), 9)

In [2]:
import sys
SRC_PATH = "/content/rabbit-classifier"
sys.path.insert(0, SRC_PATH)

In [4]:
from src.bootstrap import bootstrap, TrainerFactory
from src.data.data_handler import BasicModel
from pathlib import Path
from pytorch_lightning.loggers import WandbLogger
import wandb
import pytorch_lightning as pl
import torch
from torchmetrics import Accuracy
import torch.nn.functional as F

trainer_factory = TrainerFactory(strategy=None, max_epochs=50, logger=WandbLogger(project="rabbit-classifier"))

bootstrap(trainer_factory=trainer_factory, data_manfiest_path=Path("/content/rabbit-classifier/resources/data.csv"), model=BasicModel())

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/loggers/wandb.py:346: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  "There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse"
Global seed set to 42
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name           | Type            | Params
----------------------------------------------------
0  | layer_1        | Conv2d          | 75    
1  | layer_2        | MaxPool2d       | 0     
2  | layer_3        | ReLU            | 0     
3  | layer_4        | Flatten         | 0     
4  | layer_5        | Linear          | 75.0 K
5  | softmax_layer  | LogSoftmax      | 0     
6  | train_acc      | Accu

Training: 0it [00:00, ?it/s]

/content/rabbit-classifier/src/data/data_handler.py:288: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  result = self.softmax_layer(x_5)


Validation: 0it [00:00, ?it/s]

/content/rabbit-classifier/src/data/data_handler.py:288: UserWarning:

Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:727: UserWarning:

Detected KeyboardInterrupt, attempting graceful shutdown...



ValueError: ignored